In [1]:
# Imports

# Talk about the category (majority missing values)

import os
import datetime
import pandas as pd
import numpy as np
import scipy.stats as stats
import pingouin as pg

import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from wristpy.core import computations
from wristpy.processing import metrics, analytics
from xgboost import XGBRegressor
from dataclasses import dataclass

pd.set_option("display.max_rows", 64)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 128)

LOCAL = "/Users/michaellu/Desktop/Fall_2024/DATA1030/InternetUseClassification/data"
KAGGLE = "/kaggle/input/child-mind-institute-problematic-internet-use"

**EDA**

In [2]:
# Read in provided train dataset
train_df = pd.read_csv(f"{LOCAL}/train.csv")
train_df

,id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0
1,000fd460,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0
2,00105258,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0
3,00115b9f,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,NaN,60.0,97.0,117.0,Summer,6.0,9.0,37.0,Summer,18.0,1.0,NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,7.0,0.0,7.0,1.0,Summer,3.0,3.84191,18.2943,1131.430,1923.44,15.59250,62.7757,14.0740,4.22033,18.82430,2.0,30.4041,16.77900,58.9338,26.4798,45.9966,NaN,NaN,Winter,2.451,Summer,4.0,2.0,4.0,0.0,5.0,1.0,0.0,3.0,2.0,2.0,3.0,0.0,3.0,0.0,0.0,3.0,4.0,3.0,4.0,1.0,44.0,Summer,31.0,45.0,Winter,0.0,1.0
4,0016bb22,Spring,18,1,Summer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,Fall,13,0,Spring,60.0,Fall,16.362460,59.5,82.4,NaN,71.0,70.0,104.0,NaN,NaN,NaN,NaN,Fall,16.0,0.0,18.0,1.0,19.9,2.0,10.0,1.0,8.0,1.0,9.0,1.0,12.0,1.0,Fall,3.0,4.52277,16.3642,1206.880,2051.70,19.46110,70.8117,14.0629,2.30138,11.58830,1.0,33.3709,17.97970,66.2889,29.7790,52.8320,NaN,NaN,Winter,3.260,Winter,3.0,3.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,2.0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,32.

In [3]:
# High level overview of the dataset
print(f"There are {len(train_df.columns)} columns in the dataset.")
print(f"There are {len(train_df.id.unique())} unique ids in the dataset.")
train_df.describe()

There are 82 columns in the dataset.
There are 3960 unique ids in the dataset.


,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-PAQ_A_Total,PAQ_C-PAQ_C_Total,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
count,3960.000000,3960.000000,2421.000000,3022.000000,3027.000000,3076.000000,898.000000,2954.000000,2967.000000,2954.000000,743.000000,740.000000,740.000000,2322.000000,2282.000000,1074.000000,1062.000000,1074.000000,1063.000000,2310.000000,2271.000000,2305.000000,2267.000000,2307.000000,2269.000000,2324.000000,2285.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.000000,1991.00000,1991.000000,1991.000000,1991.000000,475.000000,1721.000000,2733.000000,2734.000000,2731.000000,2731.000000,2729.000000,2732.000000,2729.000000,2730.000000,2730.000000,2733.000000,2734.000000,2731.000000,2729.000000,2732.000000,2730.000000,2728.000000,2725.000000,2728.000000,2730.000000,2733.000000,2736.000000,2609.000000,2606.000000,3301.000000,2736.000000
mean,10.433586,0.372727,65.454771,19.331929,55.946713,89.038615,27.278508,69.648951,81.597236,116.983074,4.989233,7.370270,27.581081,11.259690,0.476337,22.420438,1.829567,23.518622,1.904045,5.579654,0.330251,8.694924,0.618880,8.805635,0.620097,9.252775,0.785558,2.651431,6.719826,19.367048,1237.018187,2064.693747,20.825346,74.021708,15.030554,4.336495,16.855020,1.745354,33.173380,20.02299,67.301883,34.389466,53.998726,2.178853,2.589550,2.370655,2.177762,2.399854,0.839253,2.297545,1.063690,0.586295,1.246520,1.062637,1.304793,1.685443,0.244599,1.340051,1.035505,1.499634,1.452346,1.627890,1.613636,1.158974,0.943652,27.896199,41.088923,57.763622,1.060588,0.580409
std,3.574648,0.483591,22.341862,5.113934,7.473764,44.569040,5.567287,13.611226,13.665196,17.061225,2.014072,3.189662,17.707751,11.807781,0.499549,10.833995,0.612585,11.148951,0.612344,7.390161,0.470407,3.429301,0.485769,3.422167,0.485469,2.988863,0.410525,1.028267,92.586325,5.047848,1872.383246,2836.246272,73.266287,199.433753,5.792505,6.356402,199.372119,0.680635,56.272346,70.21561,108.705918,84.050607,129.362539,0.849476,0.783937,1.673312,1.697117,1.588807,1.195601,1.705218,1.268282,1.049355,1.342582,1.258797,1.331715,1.543074,0.522956,1.411156,1.301712,1.492929,1.495600,1.445622,1.529178,1.343661,1.185460,20.338853,10.427433,13.196091,1.094875,0.771122
min,5.000000,0.000000,25.000000,0.000000,33.000000,0.000000,18.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-7.789610,0.048267,813.397000,1073.450000,1.789450,28.900400,7.864850,-194.163000,-8745.080000,1.000000,14.489000,4.63581,23.620100,4.655730,20.589200,0.660000,0.580000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000,38.000000,0.000000,0.000000
25%,8.000000,0.000000,59.000000,15.869350,50.000000,57.200000

In [4]:
# Understanding PCIAT-PCIAT_Total vs sii
for value in train_df.sii.unique():
    print(f"Unique value: {value}")
    print(train_df[train_df.sii == value]["PCIAT-PCIAT_Total"].describe())
    print("="*32)

# 0 -> 0 <= PCIAT-PCIAT_Total <= 30
# 1 -> 30 < PCIAT-PCIAT_Total < 50
# 2 -> 50 <= PCIAT-PCIAT_Total < 80
# 3 -> 80 <= PCIAT-PCIAT_Total

# Instead of making this a classification problem, we can make this a regression
# problem by predicting PCIAT-PCIAT_Total. This might make it more sensitive or
# accurate.

Unique value: 2.0
count    378.000000
mean      60.658730
std        7.787458
min       50.000000
25%       54.000000
50%       59.000000
75%       66.000000
max       79.000000
Name: PCIAT-PCIAT_Total, dtype: float64
Unique value: 0.0
count    1594.000000
mean       13.929737
std        10.331241
min         0.000000
25%         3.000000
50%        15.000000
75%        23.000000
max        30.000000
Name: PCIAT-PCIAT_Total, dtype: float64
Unique value: 1.0
count    730.000000
mean      38.726027
std        5.390029
min       31.000000
25%       34.000000
50%       38.000000
75%       43.000000
max       49.000000
Name: PCIAT-PCIAT_Total, dtype: float64
Unique value: nan
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: PCIAT-PCIAT_Total, dtype: float64
Unique value: 3.0
count    34.000000
mean     85.911765
std       4.230966
min      80.000000
25%      81.250000
50%      87.000000
75%      89.000000
max      93.000000
Name: 

In [5]:
# Read in data descriptions
data_descriptions = pd.read_csv(f"{LOCAL}/data_dictionary.csv")

# Non-categorical columns
cont_columns = []
cat_columns = ["sii"]

verbose = False

# Plot pie charts/histograms (large output)
filled_train_df = train_df.fillna("nan")
for _, row in data_descriptions.iterrows():
    if verbose:
        print(f"Instrument: {row.Instrument}")
        print(f"Field: {row.Field}")
        print(f"Description: {row.Description}")
        print(row)

    if row.Type == "float" or row.Type == "int":
        cont_columns.append(row.Field)
        if verbose:   
            fig = plt.figure(figsize=(10, 8))
            plt.hist(train_df[row.Field], bins=40)
            plt.xlabel(row.Field)
            plt.ylabel("Frequency")
            plt.title(f"{row.Field} Histogram") 
            plt.show()

    if row.Type == "categorical int":
        cat_columns.append(row.Field)
        if verbose:
            value_counts = filled_train_df[row.Field].value_counts()
            fig, ax = plt.subplots()
            wedges, texts, autotexts = ax.pie(
                value_counts,
                labels=value_counts.index,
                autopct='%1.1f%%',
                startangle=90
            )
            plt.title(f"{row.Field} Pie Chart")  
            plt.show()

    if row.Type == "str" and row.Values is not np.nan:
        cat_columns.append(row.Field)
        if verbose:
            value_counts = filled_train_df[row.Field].value_counts()
            fig, ax = plt.subplots()
            wedges, texts, autotexts = ax.pie(
                value_counts,
                labels=value_counts.index,
                autopct='%1.1f%%',
                startangle=90
            )
            plt.title(f"{row.Field} Pie Chart")
            plt.show()

    if verbose: print("="*32)

In [6]:
# Noted outliers in "CGAS-CGAS_Score" and some other numerical columns
print("CGAS-CGAS_Score percentiles:\n", train_df["CGAS-CGAS_Score"].quantile([0.25, 0.5, 0.75]))
values = train_df["CGAS-CGAS_Score"].to_list()
values.sort(reverse=True)
print(f"Top 5 largest values: {values[:5]}")

# We don't know if the 999.0 should be 99.0 so lets just drop that one row
train_df = train_df[train_df["CGAS-CGAS_Score"] <= 100.0]

# Re-running the cell above confirms that there was only one outlier row

CGAS-CGAS_Score percentiles:
 0.25    59.0
0.50    65.0
0.75    75.0
Name: CGAS-CGAS_Score, dtype: float64
Top 5 largest values: [nan, 78.0, 75.0, 71.0, 71.0]


In [7]:
# Correlation analysis (all train data)
cont_train_df = train_df[cont_columns]
correlation_matrix = cont_train_df.corr()
correlation_matrix["PCIAT-PCIAT_Total"].sort_values(key=lambda x: abs(x), ascending=False)

PCIAT-PCIAT_Total               1.000000
Physical-Height                 0.418913
Basic_Demos-Age                 0.410084
Physical-Weight                 0.355298
Physical-Waist_Circumference    0.349236
FGC-FGC_CU                      0.284946
Physical-BMI                    0.244067
BIA-BIA_BMI                     0.243630
SDS-SDS_Total_T                 0.231507
SDS-SDS_Total_Raw               0.226605
FGC-FGC_PU                      0.179003
FGC-FGC_TL                      0.146879
Physical-Systolic_BP            0.144060
FGC-FGC_GSD                     0.142197
FGC-FGC_GSND                    0.124995
BIA-BIA_FFMI                    0.105888
FGC-FGC_SRL                    -0.096154
FGC-FGC_SRR                    -0.084136
BIA-BIA_FMI                     0.079279
CGAS-CGAS_Score                -0.070542
BIA-BIA_LST                     0.067283
Physical-Diastolic_BP           0.064523
Fitness_Endurance-Time_Mins    -0.049669
PAQ_A-PAQ_A_Total              -0.049212
BIA-BIA_SMM     

In [8]:
# PCA on continuous columns
cont_columns_filled = cont_train_df.fillna(cont_train_df.mean())
if "PCIAT-PCIAT_Total" in cont_columns: cont_columns.remove("PCIAT-PCIAT_Total")
X = cont_columns_filled[cont_columns]
y = cont_columns_filled["PCIAT-PCIAT_Total"]

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(n_components=15)
X_train = pca.fit_transform(X)
print("PCA variance ratios: ")
for i, x in enumerate(pca.explained_variance_ratio_):
    print(
        f"  Component {i + 1} accounts {round(x * 100, 4)}% of the variance"
    )

PCA variance ratios: 
  Component 1 accounts 29.724% of the variance
  Component 2 accounts 12.647% of the variance
  Component 3 accounts 6.988% of the variance
  Component 4 accounts 5.8125% of the variance
  Component 5 accounts 5.1177% of the variance
  Component 6 accounts 4.7637% of the variance
  Component 7 accounts 4.3288% of the variance
  Component 8 accounts 3.8899% of the variance
  Component 9 accounts 3.6676% of the variance
  Component 10 accounts 2.7284% of the variance
  Component 11 accounts 2.6903% of the variance
  Component 12 accounts 2.5654% of the variance
  Component 13 accounts 2.4978% of the variance
  Component 14 accounts 2.3457% of the variance
  Component 15 accounts 2.1754% of the variance


In [9]:
# Welch's ANOVA hypothesis test between categorical columns and
# PCIAT-PCIAT_Total
cat_to_use = []
hypothesis_threshold = 0.05
for col in cat_columns:
    # Skip PCIAT columns since those are not availabe in the test dataset
    if "PCIAT" in col or "sii" in col: continue
    print(f"\nColumn: {col}")
    aov = pg.welch_anova(dv="PCIAT-PCIAT_Total", between=col, data=train_df)
    print(aov)
    if hypothesis_threshold >= aov["p-unc"].values[0]:
        print("\033[91mReject null hypothesis\033[0m")
        cat_to_use.append(col)
    else:
        print("\033[92mFail to reject null hypothesis\033[0m")
    print("="*32)


Column: Basic_Demos-Enroll_Season
                      Source  ddof1       ddof2         F     p-unc       np2
0  Basic_Demos-Enroll_Season      3  1271.71908  2.487914  0.058993  0.002986
Fail to reject null hypothesis

Column: Basic_Demos-Sex
            Source  ddof1        ddof2          F     p-unc       np2
0  Basic_Demos-Sex      1  1860.003257  23.003635  0.000002  0.009269
Reject null hypothesis

Column: CGAS-Season
        Source  ddof1        ddof2         F     p-unc       np2
0  CGAS-Season      3  1283.908405  0.467564  0.704955  0.000598
Fail to reject null hypothesis

Column: Physical-Season
            Source  ddof1        ddof2         F     p-unc       np2
0  Physical-Season      3  1207.378454  2.111131  0.097061  0.002934
Fail to reject null hypothesis

Column: Fitness_Endurance-Season
                     Source  ddof1       ddof2         F     p-unc       np2
0  Fitness_Endurance-Season      3  630.235398  2.004408  0.112124  0.041608
Fail to reject null hypoth

In [18]:
# Parquet timeseries EDA
train_df = pd.read_csv(f"{LOCAL}/train.csv")

target_dict = {}
for _, row in train_df.iterrows():
    target_dict[row["id"]] = row["PCIAT-PCIAT_Total"]

# for file_name in tqdm(os.listdir(f"{LOCAL}/series_train.parquet")):
#     if file_name.startswith("id="):
#         file_id = file_name.split("=")[1]
#         if file_id not in target_dict:
#             print(f"File {file_id} not in target_dict")

def read_timeseries(file_name, dir_name):
    df = pd.read_parquet(
        os.path.join(dir_name, file_name, "part-0.parquet"),
        "fastparquet",
    )
    # Convert nanosecond input to time of day in hours (24)
    df["daytime"] = df["time_of_day"].apply(lambda x: x / 3_600_000_000_000)
    df["id"] = file_name.split("=")[1]
    df["PCIAT-PCIAT_Total"] = target_dict.get(df["id"].values[0], np.nan)
    return df.drop("step", axis=1)

results = []
for file_name in tqdm(os.listdir(f"{LOCAL}/series_train.parquet")):
    if file_name.startswith("id="):
        results.append(read_timeseries(file_name, f"{LOCAL}/series_train.parquet"))
    if len(results) > 100:
        break

timeseries_df = pd.concat(results)
timeseries_df

# Use enmo, anglez, non-wear_flag, light, weekday, quarter, and daytime to try
# to predict PCIAT-PCIAT_Total

 10%|█         | 100/997 [00:16<02:31,  5.94it/s]


,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT,daytime,id,PCIAT-PCIAT_Total
0,-0.468869,0.412020,-0.236458,0.042506,-19.824650,0.0,27.666666,4179.000000,57480000000000,4,1,28.0,15.966667,0d01bbf2,11.0
1,-0.662526,0.533484,0.064034,0.052847,4.300246,0.0,12.666667,4178.666504,57485000000000,4,1,28.0,15.968056,0d01bbf2,11.0
2,-0.611384,0.227252,-0.150882,0.060734,-16.545208,0.0,47.000000,4178.333496,57490000000000,4,1,28.0,15.969444,0d01bbf2,11.0
3,-0.385799,0.552782,-0.500523,0.070440,-36.452175,0.0,63.799999,4178.000000,57495000000000,4,1,28.0,15.970833,0d01bbf2,11.0
4,0.016133,0.031981,-0.825109,0.081058,-67.488388,0.0,6.000000,4177.666504,57500000000000,4,1,28.0,15.972222,0d01bbf2,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412471,-0.017748,-0.015573,-0.995489,0.000032,-88.687927,1.0,1.000000,3098.833252,28055000000000,1,4,45.0,7.793056,807d8908,38.0
412472,-0.018034,-0.015482,-0.995463,0.000004,-88.682755,1.0,1.000000,3098.666748,28060000000000,1,4,45.0,7.794444,807d8908,38.0
412473,-0.017800,-0.015625,-0.995515,0.000013,-88.698128,1.0,1.000000,3098.500000,28065000000000,1,4,45.0,7.795833,807d8908,38.0
412474,-0.018021,-0.015625,-0.995489,0.000016,-88.669754,1.0,1.000000,3098.333252,28070000000000,1,4,45.0,7.797222,807d8908,38.0


In [11]:
# Analyze values from a sample of data
watch_on = timeseries_df[timeseries_df["non-wear_flag"] == 1.0]
weekdays = watch_on[watch_on.weekday.isin([1, 2, 3, 4, 5])]
weekends = watch_on[watch_on.weekday.isin([6, 7])]
print(
    "Percent of watch on samples from weekdays and weekends:",
    round(len(weekdays) / len(watch_on), 4) * 100, "% and ",
    round(len(weekends) / len(watch_on), 4) * 100, "%\n"
)
# Roughly the expected distribution of watch-on samples

print("Weekday vs weekend enmo quantiles [25, 50, 75], mean, min, and max:")
print("Weeday: ", weekdays["enmo"].describe())
print("\nWeekend: ", weekends["enmo"].describe())

Percent of watch on samples from weekdays and weekends: 70.08 % and  29.92 %

Weekday vs weekend enmo quantiles [25, 50, 75], mean, min, and max:
Weeday:  count    7.577323e+06
mean     5.635524e-03
std      1.151666e-02
min      0.000000e+00
25%      2.487500e-07
50%      4.326650e-04
75%      4.153432e-03
max      5.874002e-02
Name: enmo, dtype: float64

Weekend:  count    3.235280e+06
mean     5.130747e-03
std      1.036747e-02
min      0.000000e+00
25%      8.319616e-06
50%      7.021872e-04
75%      4.219659e-03
max      5.884418e-02
Name: enmo, dtype: float64


In [36]:
import re

def extract_seconds_from_lights_on(log_lines):
    """
    Extracts the seconds from the timestamp for log entries containing "PhoneStatusBar: setLightsOn(true)".

    Args:
        log_lines: A list of log lines.

    Returns:
        A list of seconds (as strings) corresponding to the matching log entries.
        Returns an empty list if no matching entries are found. 
    """
    seconds_list = []
    pattern = r"\d{2}:\d{2}:(\d{2})\.\d{3}.*PhoneStatusBar: setLightsOn\(true\)"

    for line in log_lines:
        match = re.search(pattern, line)
        if match:
            seconds = match.group(1)
            seconds_list.append(seconds)

    if len(seconds_list) == 0:
        print("No matching log entries found.")
    return seconds_list




# Example usage (replace with your actual log file reading):
log_lines = [
    "03-17 16:13:55.875  1702  2250 W ActivityManager: getRunningAppProcesses: caller 10091 does not hold REAL_GET_TASKS; limiting output",
    "03-17 16:13:56.123  1234  4567 I PhoneStatusBar: setLightsOn(true)",  # Match
    "03-17 16:14:01.456  7890  1234 V AudioManager: getLastAudibleStreamVolume  treamType: 1",
    "03-17 16:14:02.789  4321  8765 W PhoneStatusBar: setLightsOn(false)", #Doesn't Match
    "03-17 16:14:05.999 1111 2222 I PhoneStatusBar: setLightsOn(true)" #Match
]


extracted_seconds = extract_seconds_from_lights_on(log_lines)
print(extracted_seconds)

['56', '05']
